# Размещение баннеров

Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив с этой страницы.

Нас будет интересовать файл `checkins.dat`. Открыв его, увидим следующую структуру:

```
id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...
```

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

```
id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...
```

С помощью `pandas` построим DataFrame и убедимся, что все 396634 строк с координатами считаны успешно.

In [22]:
import pandas as pd
import numpy as np

In [2]:
with open('datasets/checkins.dat') as input_file, open('datasets/checkins.csv', 'w') as output_file:
    for line in input_file:
        new_line = [data.strip() for data in line.split('|')]
        new_line = list(filter(None, new_line))
        if len(new_line) == 6:
            output_file.write(','.join(new_line) + '\n')

In [3]:
data = pd.read_csv('datasets/checkins.csv')

In [4]:
data.shape

(396634, 6)

In [5]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом `MeanShift`, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте `MeanShift`, указав `bandwidth=0.1`, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

**Примечание**: на 396632 строках, кластеризация будет работать долго. Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [6]:
from sklearn.cluster import MeanShift

In [7]:
reduced_data = data[:100000]
ms = MeanShift(bandwidth=0.1)

In [10]:
%%time
ms.fit(reduced_data[['latitude', 'longitude']])


CPU times: user 2min 35s, sys: 274 ms, total: 2min 35s
Wall time: 2min 35s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [78]:
cluster_centers = ms.cluster_centers_
labels = ms.labels_
cluster_centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [  38.891565  , -121.2930079 ],
       [  42.5953378 ,  -78.9411461 ],
       [  41.5822716 ,  -85.8344383 ]])

In [79]:
len(cluster_centers)

3230

Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт `mapcustomizer.com` имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

```
38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...
```

In [80]:
from collections import Counter

counter_labels = Counter(labels)
cluster_labels = [key for key, value in counter_labels.items() if value > 15]
len(cluster_labels)

592

In [81]:
cluster_centers = cluster_centers[cluster_labels, :]

In [37]:
len(cluster_centers)

592

In [39]:
cluster_centers

array([[  38.88616522,  -77.04878333],
       [  33.76663623,  -84.39328918],
       [  45.52348321, -122.67628042],
       ...,
       [  50.1115118 ,    8.6805059 ],
       [  42.0166667 ,  -94.3766667 ],
       [  37.2046429 ,  -80.4126892 ]])

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

```
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)
```

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

**Примечание**: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико, чтобы не повлиять на кластеризацию или сортировку.

In [38]:
offices = [(33.751277, -118.188740), (25.867736, -80.324116), (51.503016, -0.075479),
          (52.378894, 4.885084), (39.366487, 117.036146), (-33.868457, 151.205134)]

In [40]:
def euclid(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [82]:
distances = []
for cluster in cluster_centers:
    min_dist = float('inf')
    for office in offices:
        dist = euclid(cluster, office)
        min_dist = min(min_dist, dist)
    distances.append(min_dist)

In [83]:
distances = np.array(distances).reshape(len(distances), 1)
cluster_centers = np.hstack((cluster_centers, distances))
cluster_centers

array([[  38.88616522,  -77.04878333,   13.42413138],
       [  33.76663623,  -84.39328918,    8.885426  ],
       [  45.52348321, -122.67628042,   12.59852603],
       ...,
       [  50.1115118 ,    8.6805059 ,    4.42111405],
       [  42.0166667 ,  -94.3766667 ,   21.40705827],
       [  37.2046429 ,  -80.4126892 ,   11.3372529 ]])

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [91]:
closest_centers = sorted(cluster_centers, key=lambda x: x[2])[:20]
closest_centers

[array([-3.38606304e+01,  1.51204776e+02,  7.83475816e-03]),
 array([5.23729640e+01, 4.89231722e+00, 9.35331619e-03]),
 array([ 2.58456723e+01, -8.03188906e+01,  2.26740662e-02]),
 array([ 5.15029913e+01, -1.25537289e-01,  5.00582948e-02]),
 array([ 3.38098780e+01, -1.18148924e+02,  7.08477324e-02]),
 array([ 25.78581242, -80.21793804,   0.13410903]),
 array([ 25.70534972, -80.28342874,   0.16740596]),
 array([ 26.01009825, -80.19999059,   0.18887596]),
 array([  33.88832534, -118.04892817,    0.19577946]),
 array([  33.87298601, -118.36209115,    0.21181054]),
 array([  33.97257482, -118.16837067,    0.22223329]),
 array([ 26.13884379, -80.33434684,   0.27130076]),
 array([  33.98393587, -118.00740497,    0.29497889]),
 array([ 26.12086266, -80.15890668,   0.30227012]),
 array([  33.81730643, -117.89124917,    0.3047305 ]),
 array([  34.06039755, -118.24870903,    0.31488379]),
 array([  33.67430266, -117.85878927,    0.33881047]),
 array([ 26.20058464, -80.25071613,   0.34084565]),
 

In [94]:
closest_lat, closest_lon = closest_centers[0][0], closest_centers[0][1]
with open('closest_center.txt', 'w') as f:
    f.write(str(closest_lat) + ' ' + str(closest_lon))

In [95]:
closest_lat, closest_lon

(-33.86063042857143, 151.20477592857145)